In [7]:
import pandas as pd
import scipy as sp

In [5]:
for country in ['Egypt', 'Poland', 'Romania']:
    lend_path = 'Total Loans YoY Monthly ' + country + '.csv'
    lend = pd.read_csv(lend_path)
    lend = lend.iloc[26:]
    lend_name = 'Total Loans: YoY: Monthly: ' + country
    lend.rename(columns={'Unnamed: 0': 'Date', lend_name: 'Bank Lending yryr%'}, inplace = True)
    lend['Date'] = pd.to_datetime(lend['Date']) + pd.tseries.offsets.MonthEnd(0)
    lend['Bank Lending yryr%'] = lend['Bank Lending yryr%'].astype(float)

    csv_path = 'Bank_Lending_yryr%_' + country + '.csv'
    lend.to_csv(csv_path, index = False)

In [22]:
lend_path = 'Total Loans YoY Quarterly Hungary.csv'
lend = pd.read_csv(lend_path)
lend = lend.iloc[32:]
lend_name = 'Total Loans: YoY: Quarterly: Hungary'
lend.rename(columns={'Unnamed: 0': 'Date', lend_name: 'Bank Lending Quarterly yryr%'}, inplace = True)
lend['Date'] = pd.to_datetime(lend['Date']) + pd.tseries.offsets.MonthEnd(0)
lend['Bank Lending Quarterly yryr%'] = lend['Bank Lending Quarterly yryr%'].astype(float)
lend = lend[lend.index.values % 3 == 2]
f = sp.interpolate.CubicSpline(lend['Date'], lend['Bank Lending Quarterly yryr%'], bc_type = 'natural')
start = '2008-01'
end = lend['Date'].iloc[-1] + pd.DateOffset(months = 1)
full = pd.date_range(start, end, freq = "M").to_series().reset_index(drop = True)
lendi = pd.Series(f(full))
datai = pd.DataFrame({"Date": full, "Bank Lending yryr%": lendi})

datam = pd.merge(datai, lend, on = 'Date', how = 'left')[['Date', 'Bank Lending yryr%']]
csv_path = 'Bank_Lending_yryr%_Hungary.csv'
datam.to_csv(csv_path, index = False)


In [21]:
datam

,Date,Bank Lending yryr%,Bank Lending Quarterly yryr%
0,2008-01-31,8.930761,8.930761
1,2008-02-29,7.675969,NaN
2,2008-03-31,6.560289,NaN
3,2008-04-30,5.927111,5.927111
4,2008-05-31,5.876486,NaN
...,...,...,...
179,2022-12-31,0.486631,NaN
180,2023-01-31,0.353224,0.353224
181,2023-02-28,0.080198,NaN
182,2023-03-31,-0.404159,NaN
